In [ ]:
import cv2
import numpy as np
import os
import glob
import peakutils
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
Test file classes:

Misclassified images in the last run: {1,2,7,8,9,10,11,14,17,20,21,22,26,29,30,31,32}

Images containing weed lagoons: {1,2,

Should you focus only on the inner quadrants to the detected lines and discard the outer ones? Since they always give problems for the perspective

In [ ]:
Problems:

Detection of weed lagoons
Angles not possible on straight lines
Add control statements to discard images with error detection (lines intersecting in the image)
Is it necessary to add width to the crop? If larger crop widths are taken, this should not be necessary.

In [ ]:
Some of the misclassified images should be discarded.
For the next one you have to save the information of angle of shot, width of cultivation and height. A skeleton must be assembled to keep these during the shot.

Image 2 shows a lagoon but of cultivation. It presents weeds on the right side but this lagoon gives problems to detect lines with Houghs, one possibility is to cut the image since it is on the right, to generalize, the side (left or right) that presents a lagoon would be cut, if the other side does not present gaps it is analyzed, but the whole image is declared with the presence of weeds.

In image 1 the curves intersect because there is a lot of vegetation mixed on one side, the microROIs are crossed between furrows.

In image 4 it does not detect certain weeds, it requires adjustments (possibly in the percentage to classify as weed)

In [ ]:
#Miscellaneous support functions

In [ ]:
# Para mostrar una imagen
def mostrar_img(lbl, img, reduction_ratio=0):
    mostrar_imgs([lbl],[img], reduction_ratio)

# Para mostrar varias imágenes
def mostrar_imgs(lbls, vec_img, reduction_ratio=0):
    if ((reduction_ratio >= 1) or (reduction_ratio < -1)):
        reduction_ratio = 0
    for i in range(len(vec_img)):
        h = vec_img[i].shape[0]
        h = (int)(h - (h*reduction_ratio))
        w = vec_img[i].shape[1]
        w = (int)(w - (w*reduction_ratio))
        img_rs = cv2.resize(vec_img[i], (w, h))
        cv2.imshow(lbls[i], img_rs)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
###Para escalar la imagen de 0 a max_value
def scaling(img, max_value):
    amax = np.amax(img)
    amin = np.amin(img)
    scalingIndex = max_value/(amax-amin)
    img = ((img-amin)*scalingIndex)
    img = img.astype(np.uint8)
    return img

### Devuelve el porcentaje de vegetación que hay en la imágen (255 es vegetación)
def get_total_vegetation(binary_img):
    return (np.sum(binary_img == 255) / (binary_img.shape[0] * binary_img.shape[1]))

### Busca los puntos máximos de una distribución (Buscar librerías en C++)
def get_maximum_points(img, winSize):
    #Generación de histograma
    histogram = column_histogram(img)
    #Suavizado de histograma
    min_dist = int(winSize/2)
    winMidSize = int(min_dist/2)
    smoothed = np.zeros(histogram.shape, np.int16)
    for i in range(0, len(histogram)):
        total = 0
        for j in range(0, min_dist):
            tmp = i - winMidSize + j
            if (tmp >= 0 and  tmp < len(histogram)):
                total += histogram[tmp]
        smoothed[i] = int(total/min_dist)
    #Detección de máximos locales
    indexes = []
    min_dist = winSize
    meceta_counter = 0
    local_maximas = 0
    last_max_dist = 0
    i=0
    while(i<len(histogram)):
        if(local_maximas == 0 and smoothed[i]>0):
            indexes.append(i)
#             print('agregado: ' + str(i))
            local_maximas += 1
            last_max_dist=0
        else:
            if(local_maximas!=0 and smoothed[i]>0):
#                 if(indexes[local_maximas-1]>480):
#                   2, 94, 1, 12
#                     print(str(local_maximas) + ', ' + str(i) + ', ' + str(last_max_dist) + ', ' + str(meceta_counter))
                if(smoothed[i]>smoothed[indexes[local_maximas-1]]): #Es mayor
                    if(last_max_dist<=min_dist):
                        indexes[local_maximas-1]=i
                        last_max_dist=0
                    else:
                        indexes.append(i)
#                         print('agregado: ' + str(i))
                        local_maximas += 1
                        last_max_dist=0
                else:
                    if(smoothed[i]<smoothed[indexes[local_maximas-1]]):
                        if(smoothed[i]>smoothed[i-1] and last_max_dist>min_dist):
                            indexes.append(i)
#                             print('agregado: ' + str(i))
                            local_maximas += 1
                            last_max_dist=0
                    else:
                        if(smoothed[i]==smoothed[indexes[local_maximas-1]] and i==indexes[local_maximas-1]+1):
                            meceta_counter=0
                            while(i<len(histogram)-1 and smoothed[i]==smoothed[indexes[local_maximas-1]]):
                                i+=1
                                meceta_counter+=1
                                last_max_dist+=1
                            if(smoothed[i]<smoothed[i-1]):
                                _tmp = indexes[local_maximas-1]+int(meceta_counter/2)
                                if (_tmp-indexes[local_maximas-1]>min_dist):
                                    indexes.append(_tmp)
#                                     print('agregado: ' + str(_tmp))
                                    local_maximas+=1
                                    last_max_dist=0
                                
        i+=1
        last_max_dist+=1

    #Gráfica
    indexes = np.array(indexes, np.int16)
    #------------------------#
    if (debugging):
        x = np.arange(0, img.shape[1], 1)
        fig, ax = plt.subplots()
        ax.plot(x, smoothed)
        ax.scatter(indexes, smoothed[indexes], c='blue')
        ax.grid()
        plt.show()
    if(store):
        fig.savefig('maximos.png')
    #------------------------#
    return indexes

# Devuelve 1 si la posición enviada pertenece al cultivo, -1 si pertenece a la maleza
def is_crop_or_furrow_lineal(lines, available, p0, crop_width):
    # Crop: 1, Crop furrow: -1, None: 0¿
    total_active = np.sum(available)
    if (total_active != 0):
        x, y = p0
        h_buffer = {}
        flag_ml = False
        flag_mr = False
        for line_i in range(0, len(lines)):
            if (available[line_i]):
                h_buffer[line_i] = lines[line_i].get_y(x)
                c_left = h_buffer[line_i] - crop_width
                c_right = h_buffer[line_i] + crop_width
                if (y <= c_right and y >= c_left): #Si está dentro de alguna franja de cultivo, es surco
                    return 1
                if (flag_ml):
                    most_left = min(most_left, h_buffer[line_i] - crop_width)
                else:
                    most_left = h_buffer[line_i] - crop_width
                    flag_ml = True
                if(flag_mr):
                    most_right = max(most_right, h_buffer[line_i] + crop_width)
                else:
                    most_right = h_buffer[line_i] + crop_width
                    flag_mr = True
        if (y <= most_left or y >= most_right): #Caso a izquierda de primera linea o a derecha de última línea
            return 0 
        return -1 #Si no es ninguno de los otros, el entre surco
    else:
        return 0

# Escalar los valores para que no tenga overflow, ver como modifican los escalares
def least_squares_line(x_list, y_list):
    # Calcular el centroide en vez de usar point0
    xc, yc = np.sum(x_list)/len(x_list), np.sum(y_list)/len(y_list)
    x_list, y_list = np.array(x_list, np.uint32), np.array(y_list, np.uint32)
    sum_x, sum_y = np.sum(x_list), np.sum(y_list)
    sum_xy = np.sum(np.multiply(x_list, y_list))
    sum_x_squared = np.sum(np.multiply(x_list, x_list))
    numerator = x_list.shape[0] * sum_xy - (sum_x * sum_y)
    denominator = x_list.shape[0] * sum_x_squared - (sum_x * sum_x)
    # Si 0 es denominador, es una recta vertical, la pendiente es infinito
    m = numerator / denominator# if denominator != 0 else 999 
    b = int(yc - m * xc)
    return m, b  # Coeficientes: Lineal y independiente

### Promedio de una lista de valores
def mean(values_list):
    values_list = np.array(values_list, np.uint64)
    return (np.sum(values_list) / values_list.shape[0])

class Line:
    #Si la pendiente es mayor a 80, se toma como recta vertical y se debe usar x_offset
    def __init__(self, m, b):
        self.m = m
        self.b = b
        
    def __str__(self):
        return 'm: ' + str(self.m) + ' - b: ' + str(self.b) + ' - x_offset: ' + str(self.x_offset)# + ' - white amount: ' + str(self.white_amount)
    
    # Devuelve el valor de x en una recta, dada su altura (y), si la pendiente es muy pronunciada, se toma como recta vertical
    def get_y(self, x):
        return int(x*self.m+self.b)
    
    def __copy__(self):
        return Line(self.m, self.b)

class MicroROI:
    def __init__(self, microimg, h_offset, v_offset, possible_centroid=None):
        vert_sum, horiz_sum, _sum = 0, 0, 0
        if (possible_centroid == None):
            vc, hc = int(microimg.shape[1] / 2) + v_offset, int(microimg.shape[0] / 2) + h_offset
        else:
            vc, hc = possible_centroid
        for h in range(0, microimg.shape[1]):
            for v in range(0, microimg.shape[0]):
                if (microimg[v, h] > 0):  # Cambiar a binario
                    vert_sum, horiz_sum, _sum = vert_sum + v, horiz_sum + h, _sum + 1
        if (_sum != 0):
            vc, hc = int(vert_sum / _sum)+v_offset, int(horiz_sum / _sum)+h_offset
        self.vc, self.hc = vc, hc
        
    def get_centroid(self):
        return self.vc, self.hc

### Devuelve la imágen con el índice seleccionado aplicado
def img_to_color_index(img, index):
    if (index == 'cive'):
        return get_CIVE(img)
    elif (index == 'exg'):
        return get_ExG(img)
    else:
        return img

def get_normalized(img):
    img = img.astype(np.uint16)
    denominator = img[:, :, 0] + img[:, :, 1] + img[:, :, 2]
    # Max 255+255+255, Min 1
    denominator = np.where(denominator == 0, 1, denominator)
    return cv2.merge((img[:, :, 0]/denominator, img[:, :, 1]/denominator, img[:, :, 2]/denominator))

def get_CIVE(img):
    img_norm = get_normalized(img)
    return 0.441*img_norm[:, :, 0]-0.811*img_norm[:, :, 1]+0.385*img_norm[:, :, 2]+18.78745

def get_ExG(img):
    r, g, b = cv2.split(img)
    r_mx = np.amax(r)
    g_mx = np.amax(g)
    b_mx = np.amax(b)
    if (r_mx == 0):
        r_mx = 1
    if (g_mx == 0):
        g_mx = 1
    if (b_mx == 0):
        b_mx = 1
    r = r / r_mx
    g = g / g_mx
    b = b / b_mx
    denominator = r + g + b
    denominator = np.where(denominator == 0, 1, denominator)
    r = r / denominator
    g = g / denominator
    b = b / denominator
    return 2*g-r-b

### Devuelve el histograma de columnas (para imágenes de 8 bits de profundidad, 255 = blanco)
def column_histogram(img_in):
    vector = np.zeros(img_in.shape[1], np.uint16)
    for col in range(0, img_in.shape[1]):
        vector[col] = np.sum(img_in[:, col] > 0)
    return vector

# Para realizar un corte en el margen superior de la imagen
def v_crop_top(img, crop_ratio):
    if (crop_ratio < 0 or crop_ratio > 1):
        crop_ratio = 0
    h2 = img.shape[0]
    h1 = (int)(h2*crop_ratio)
    w = img.shape[1]
    return img[h1:h2, 0:w]

In [ ]:
Changes:
    
Verify that all variables are used, declared, etc.
Verify which is better, least absolute deviation or least squares
Make the point class for the coordinate list of centroids
Get the algorithms that use libraries, look for simple solutions with the same results

In [ ]:
#Configuration variables

In [ ]:
# Reducción del tamaño original para poder ver las imágenes en tamañas chicas
reduction_to_see = 0.4

# Kernels para utilizar
kernel_ones = np.ones((3, 3), np.uint8)
kernel_plus = np.array([[0,1,0],
                        [1,1,1],
                        [0,1,0]], np.uint8)
# kernel_verticales = np.array([[1, 0, -1],  # Segmentos verticales
#                              [2, 0, -2],
#                              [1, 0, -1]])
# kernel_vertical = np.array([
#     [0, 0, 0, 0, 0],
#     [0, 0, 1, 0, 0],
#     [0, 0, 1, 0, 0],
#     [0, 0, 1, 0, 0],
#     [0, 0, 0, 0, 0]
# ], np.uint8)
kernel_vertical = np.array([
    [0, 1, 0],
    [0, 1, 0],
    [0, 1, 0],
], np.uint8)

debugging = True
# debugging = False
store = False
# store = True
                                        #-------DEPENDIENTES-------#
#-------Depende del ángulo de cámara-------#
# Cuanto se corta de la imagen para quitar el punto de fuga
porcentaje_de_crop = 0

#-------Depende del tipo de planta y su estadio-------#
# Ancho de línea de cultivo en píxeles
crop_width_bottom = 35
crop_width_top = 10
# Este porcentaje de ancho es dependiente del tipo de cultivo y el estadío en el que se encuentra
addition_percentage = 0.6
# addition = crop_width_bottom * addition_percentage

#-------Depende de la luminosidad-------#
# Índices posibles: cive exg (cive para nublado, exg para soleado)
indice_seleccionado = 'exg'

                                        #-------AJUSTABLES-------#
# Un cuarto de la imagen original
rs_ratio = 0.25
# Porcentaje mínimo requerido para aplicar erosión y resaltar los surcos
porcentaje_para_erosion = 0.15
#Porcentaje de maleza mínimo que tiene que tener un cuadrante para detectarlo con maleza
weed_porc = 0.002


In [ ]:
#Algorithm
#Detection

In [ ]:
# %%timeit -n 10 -r 10
# 855 ms ± 80.4 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
for i in range(3, 4):
# for i in range(1, 33):
    path_to_imgs = 'img/test/originals_rs_2/'
    file_name = 'test' + str(i) + '_rs'
#     print('El nombre del archivo cargado es: ' + file_name)
    _original = cv2.imread(path_to_imgs + file_name + '.png', cv2.IMREAD_COLOR)
    if _original is None:
        print('Warning: imagen no cargada')
    else:

        #-------PREPARACIÓN DE DATOS-------#

        # Cortar la parte superior de la imagen (esto se calcula de manera manual, depende del ángulo de pitch que tenga la cámara)
        # Módulo IP de control que pérmite el paso de los píxeles por debajo
        original = _original#v_crop_top(_original, porcentaje_de_crop)

        #------------------------#
        if (debugging):
            mostrar_img('cropped', original, reduction_to_see)
        #------------------------#
    
        # Utilizar el algoritmo con un reescalado de la imagen (Se debe poder configurar el tamaño de reducción y el algortimo utilizado)
        resized = cv2.resize(original, None, fx=rs_ratio,
                             fy=rs_ratio, interpolation=cv2.INTER_AREA)
        
        # Obtención de imagen con índice
        # Para módulo IP - Debe devolver índice en 8 bits
        # Se utilizan los índice CIVE y ExG (Siendo el índice CIVE eficiente para días parcial y totalmente nublados, y el índice ExG para días soleados)
        img_veg = img_to_color_index(resized, indice_seleccionado)

        # La convierto en binario de 8 bits para aplicar Otsu (No debe ser parte del VHDL, ya debe devolver 8 bits)
        img_veg = scaling(img_veg, 255)
        # La parte de inversión de valor debe estar dentro del calculo del índice
        if (indice_seleccionado == 'exg'):
            img_veg = cv2.bitwise_not(img_veg)
            
        #------------------------#
        if (debugging):
            mostrar_img('indice', img_veg)
        if(store):
            cv2.imwrite(path_to_imgs + file_name + '_' + 'indice' + '.png', img_veg)
        #------------------------#

        # Otsu (Comienza parte pesada de algoritmo, a partir de acá debe ir en un mismo módulo IP)
        ret, segmented = cv2.threshold(
            img_veg, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        new_height = img_veg.shape[0]
        new_width = img_veg.shape[1]

        #Erosión vertical para destacar las rectas
        img_veg = cv2.erode(segmented, kernel_vertical)

        # Utilizo eroción para acentuar la diferencia entre líneas de cultivo, cuando la cantidad de vegetación es alta
        if (get_total_vegetation(img_veg) > porcentaje_para_erosion):
            img_veg_eroded = (cv2.erode(img_veg, kernel_plus))
        else:
            img_veg_eroded = img_veg.copy()

        # ---------------------- Para implementar
        # Si se detectan lagunas de vegetación (tiene que ver con el ancho de los cultivos según
        # la distancia en la imágen) se debe marcar como zona con maleza, y el resto "limpio",
        # ya que el algoritmo para detección de las rectas no va a funcionar si siguieramos

        #------------------------#
        if (debugging):
            mostrar_imgs(['segmented', 'img_veg', 'img_veg_eroded'],
                         [segmented, img_veg, img_veg_eroded])
        if(store):
            cv2.imwrite(path_to_imgs + file_name + '_' + 'segmentado' + '.png', segmented)
            cv2.imwrite(path_to_imgs + file_name + '_' + 'erosion1' + '.png', img_veg)
            cv2.imwrite(path_to_imgs + file_name + '_' + 'erosion_condicional' + '.png', img_veg_eroded)
        #------------------------#

        #-------HISTOGRAMA-------#
        # Calcular histograma de la franja inferior de la imágen, detectar la cantidad de picos según la cantidad de lineas se busquen, si hay más o menos de lo buscado se debe descartar la imagen.
        h_space = int(new_height/6)
        indexes = get_maximum_points(img_veg[(
            new_height - h_space):new_height, 0:new_width], crop_width_bottom)

        #------------------------#
        if (debugging):
            print('Los valores de las bases detectados son: ' + str(indexes))
        if (store):
            cv2.imwrite(path_to_imgs + file_name + '_' + '12th' + '.png', img_veg_eroded[(new_height - h_space):new_height, 0:new_width])
        #------------------------#

        #-------SEGMENTACIÓN-------#
        # Segmentación de la imagen en partes que disminuyen un 5% cuando se avanza hacia arriba
        # La segmentación es dependiente del alto de la imagen, ya que siempre se calculan 12, por tanto se puede realizar una vez cuando cambia el
        # valor de corte, de proporción de disminución y de cambio en resolución
        # Puede utilizarse un módulo IP aparte para que lo calcule, hay que calcular el alto de la en base a los parametros de la imagen
# -------------------- ESTATICO
        # Para obtener los 4 segmentos inferiores
        middle = new_height-int(new_height/2)
        third_1 = (int)(new_width/3)
        third_2 = (int)(new_width*(2/3))

        x_segments_bottom = np.arange(11, 15)
        x_segments_top = np.arange(0, 9)

        # Para obtener los 8 segmentos superiores
        x_segments_bottom = (np.multiply((np.true_divide(
            x_segments_bottom, np.sum(x_segments_bottom))), middle)).astype(np.uint16)
        x_segments_top = (np.multiply((np.true_divide(
            x_segments_top, np.sum(x_segments_top))), middle)).astype(np.uint16)
        x_segments_bottom = x_segments_bottom[::-1]
        x_segments_top = x_segments_top[::-1]
        x_segments_bottom[0] = x_segments_bottom[0] + \
            middle - np.sum(x_segments_bottom)
        x_segments_top[0] = x_segments_top[0] + middle - np.sum(x_segments_top)

        # Se obtienen los 12 segmentos uniendo los 4 inferiores y los 8 superiores
        x_segments = np.concatenate((x_segments_bottom[0:len(
            x_segments_bottom) - 1], x_segments_top[0:len(x_segments_top)]))
        for i in range(len(x_segments) - 2, -1, -1):
            x_segments[i] = x_segments[i] + x_segments[i+1]
        x_segments = np.concatenate(([new_height], x_segments))

        #------------------------#
        if (debugging):
            with_lines = cv2.merge(
                (img_veg_eroded, img_veg_eroded, img_veg_eroded))
            for x in x_segments:
                if x > 0:
                    if (x >= middle):
                        cv2.line(with_lines, (0, x),
                                 (with_lines.shape[1], x), (255, 0, 0), 1)
                    else:
                        cv2.line(with_lines, (0, x),
                                 (with_lines.shape[1], x), (0, 255, 0), 1)
            mostrar_img('Divisions', with_lines)
        #------------------------#

        #-------CÁLCULO DE MICROROIS-------#
        # Se calcula el centro del ROI en base a las rectas encontradas por TH
        # Si el microROI no tiene píxeles blancos se asume que el centro es el centroide
        # ---------------------- Para implementar
        # Tres vacios consecutivos en una línea es el tope, sino se descarta la imagen

        # Matrices de puntos (puede ser una clase) depende de la cantidad de rectas (cantidad dinámica, en HLS se debe poner un tope)
        x_centroids_matrix = np.zeros(
            (len(indexes), len(x_segments) - 1), np.uint16)
        y_centroids_matrix = np.zeros(
            (len(indexes), len(x_segments) - 1), np.uint16)
# -------------------- ESTATICO
        crop_width = crop_width_bottom
        crop_width_decay_seg = (crop_width_bottom - crop_width_top) / \
            (len(x_segments) - 1)  # (inicial-final)/pasos

        #------------------------#
        if (debugging):  # Se crea la imagen para posterior muestra en debugging
            with_lines = img_veg_eroded.copy()
            with_lines = cv2.merge((with_lines, with_lines, with_lines))
            print('El tamaño de la matriz de centroides es: ' +
                  str(x_centroids_matrix.shape))
        #------------------------#

        # Contador del segmento en el que se posiciona, los valores son: [1, ..., 12]
        segment_index = 1
        # Primer centroide con las bases
        for i_line in range(0, len(indexes)):
            y_left, y_right = indexes[i_line] - \
                int(crop_width/2), indexes[i_line] + int(crop_width/2)
            if (y_left < 0):
                y_left = 0
            if (y_right >= new_width):
                y_right = new_width-1
            x_bottom, x_top = x_segments[segment_index -
                                         1], x_segments[segment_index]
            centroid = MicroROI(
                img_veg_eroded[x_top:x_bottom, y_left:y_right], y_left, x_top)
            x_centroids_matrix[i_line, segment_index - 1], y_centroids_matrix[i_line,
                                                                              segment_index - 1] = centroid.get_centroid()
            #------------------------#
            if (debugging):
                cv2.line(with_lines, (y_left, x_top),
                         (y_left, x_bottom), (255, 255, 0), 1)
                cv2.line(with_lines, (y_right, x_top),
                         (y_right, x_bottom), (255, 255, 0), 1)
                cv2.circle(with_lines, (y_centroids_matrix[i_line, segment_index - 1],
                                        x_centroids_matrix[i_line, segment_index - 1]), 3, (255, 0, 255), 2)
            #------------------------#
        crop_width = crop_width - crop_width_decay_seg
        segment_index = segment_index + 1  # Próxima franja
        
        #------------------------#
        if (debugging):
            mostrar_img('Primer_c', with_lines)
        if(store):
            cv2.imwrite(path_to_imgs + file_name + '_' + 'p_c' + '.png', with_lines)
        #------------------------#

        # Segundos centroides
        for i_line in range(0, len(indexes)): 
            y_left, y_right = y_centroids_matrix[i_line, segment_index-2] - int(
                crop_width/2), y_centroids_matrix[i_line, segment_index-2] + int(crop_width/2)
            if (y_left < 0):
                y_left = 0
            if (y_right >= new_width):
                y_right = new_width-1
            x_bottom, x_top = x_segments[segment_index -
                                         1], x_segments[segment_index]
            centroid = MicroROI(
                img_veg_eroded[x_top:x_bottom, y_left:y_right], y_left, x_top)
            x_centroids_matrix[i_line, segment_index - 1], y_centroids_matrix[i_line,
                                                                              segment_index - 1] = centroid.get_centroid()
            #------------------------#
            if (debugging):
                cv2.line(with_lines, (y_left, x_top),
                         (y_left, x_bottom), (255, 255, 0), 1)
                cv2.line(with_lines, (y_right, x_top),
                         (y_right, x_bottom), (255, 255, 0), 1)
                cv2.circle(with_lines, (y_centroids_matrix[i_line, segment_index - 1],
                                        x_centroids_matrix[i_line, segment_index - 1]), 3, (255, 0, 255), 2)
            #------------------------#
        crop_width = crop_width - crop_width_decay_seg
        segment_index = segment_index + 1

        #------------------------#
        if (debugging):
            mostrar_img('Segundo_c', with_lines)
        if(store):
            cv2.imwrite(path_to_imgs + file_name + '_' + 's_c' + '.png', with_lines)
        #------------------------#

        # Centroides tercero y cuarto LEAST 2 y 3
        for i_seg in range(segment_index, 5):
            for i_line in range(0, len(indexes)):  # 5 es la mitad de la imagen
                x_list = x_centroids_matrix[i_line, 0: i_seg-1]
                y_list = y_centroids_matrix[i_line, 0: i_seg-1]
                m, b = least_squares_line(x_list, y_list)
                line = Line(m, b)
                x_bottom, x_top = x_segments[i_seg - 1], x_segments[i_seg]
                x_middle = (x_bottom-x_top)/2+x_top
                y_temp = line.get_y(x_middle)
                y_left, y_right = y_temp - \
                    int(crop_width/2), y_temp + int(crop_width/2)
                if (y_left < 0):
                    y_left = 0
                if (y_right >= new_width):
                    y_right = new_width-1

                centroid = MicroROI(
                    img_veg_eroded[x_top:x_bottom, y_left:y_right], y_left, x_top)
                x_centroids_matrix[i_line, i_seg - 1], y_centroids_matrix[i_line,
                                                                          i_seg - 1] = centroid.get_centroid()
                #------------------------#
                if (debugging):
                    cv2.line(with_lines, (y_left, x_top),
                             (y_left, x_bottom), (255, 255, 0), 1)
                    cv2.line(with_lines, (y_right, x_top),
                             (y_right, x_bottom), (255, 255, 0), 1)
                    cv2.circle(with_lines, (y_centroids_matrix[i_line, i_seg - 1],
                                            x_centroids_matrix[i_line, i_seg - 1]), 3, (255, 0, 255), 2)
                #------------------------#
            crop_width = crop_width - crop_width_decay_seg

        segment_index = 5

        #------------------------#
        if (debugging):
            mostrar_img('Mitad_c', with_lines)
        if(store):
            cv2.imwrite(path_to_imgs + file_name + '_' + 'm_c' + '.png', with_lines)
        #------------------------#

        # Usar la recta con los últimos cuatro elementos para el ROI del siguiente nivel
        # Calcula centroides del siguiente nivel y agregarlos en las matrices correspondientes
        last_centroids_index = 0
        while segment_index < (len(x_segments)):
            # Para cada línea
            for quadrant_index in range(0, x_centroids_matrix.shape[0]):
                # Tomo elementos de [i, i+3] correspondientes a los últimos cuatro centroides
                x_list = x_centroids_matrix[quadrant_index,
                                            last_centroids_index:last_centroids_index + 4]
                y_list = y_centroids_matrix[quadrant_index,
                                            last_centroids_index:last_centroids_index + 4]
#                 last_centroid = (y_list[-1],x_list[-1])
                m, b = least_squares_line(x_list, y_list)  # , last_centroid)
#                 m, b = least_squares_line(x_list, y_list)
                line = Line(m, b)
                # Inferior, superior
                x1, x2 = x_segments[segment_index -
                                    1], x_segments[segment_index]
                x_middle = (x1-x2)/2+x2
                y_temp = line.get_y(x_middle)
                y_possible = line.get_y(x_middle)
                # Desplazamientos según ancho de línea
                y1, y2 = y_temp - \
                    int(crop_width / 2), y_temp + int(crop_width / 2)
                # Pasarle microimagen con desplazamiento en x e y
                centroid = MicroROI(
                    img_veg_eroded[x2:x1, y1:y2], y1, x2, (x_middle, y_possible))
                x_centroids_matrix[quadrant_index, segment_index -
                                   1], y_centroids_matrix[quadrant_index, segment_index-1] = centroid.get_centroid()

                #------------------------#
                if (debugging):
                    cv2.line(with_lines, (y1, x1), (y1, x2), (255, 255, 0), 1)
                    cv2.line(with_lines, (y2, x1), (y2, x2), (255, 255, 0), 1)
                    cv2.circle(with_lines, (y_centroids_matrix[quadrant_index, segment_index-1],
                                            x_centroids_matrix[quadrant_index, segment_index-1]), 3, (255, 0, 255), 2)
                #------------------------#

            segment_index = segment_index + 1  # Próxima franja
            crop_width = crop_width - crop_width_decay_seg
            last_centroids_index = last_centroids_index + 1

        #------------------------#
        if (debugging):
            mostrar_img('Centroides', with_lines)
            print('x:' + str(x_centroids_matrix))
            print('y:' + str(y_centroids_matrix))
        if(store):
            cv2.imwrite(path_to_imgs + file_name + '_' + 't_c' + '.png', with_lines)
        #------------------------#

        #------MÉTODO DE MÍNIMOS CUADRADOS-------#
        # Una vez obtenidos todos los centroides de cada línea se realiza una análisis de regresión

        # La cantidad de columnas depende de la cantidad de surcos
        lines = {}
        available = np.empty(x_centroids_matrix.shape[0], np.bool)

        #------------------------#
        if (debugging):
            with_lines = img_veg_eroded.copy()
            with_lines = cv2.merge((with_lines, with_lines, with_lines))
        #------------------------#

        # Para cada línea
        for quadrant_index in range(0, x_centroids_matrix.shape[0]):
            x_list = x_centroids_matrix[quadrant_index,
                                        0:x_centroids_matrix.shape[1]]
            y_list = y_centroids_matrix[quadrant_index,
                                        0:y_centroids_matrix.shape[1]]
#             last_centroid = (y_list[-1], x_list[-1])
            m, b = least_squares_line(x_list, y_list)  # , last_centroid)
            line = Line(m, b)
            lines[quadrant_index] = line
            y_temp = line.get_y(new_height)
            if ((y_temp <= third_1 and m < -0.15 and m > -1) or (y_temp >= third_2 and m > 0.15 and m < 1) \
                    or (y_temp > third_1 and y_temp < third_2 and m < 0.4 and m > -0.4)):
                available[quadrant_index] = True
                #------------------------#
                if (debugging):
                    print('line: ' + str(quadrant_index) + ' m: ' + str(m) + ' b: ' + str(b))
                #------------------------#
            else:
                available[quadrant_index] = False
            #------------------------#
            if (debugging):
                if (available[quadrant_index]):
                    
                    x1, x2 = 0, new_height
                    y1, y2 = lines[quadrant_index].get_y(
                        x1), lines[quadrant_index].get_y(x2)
                    cv2.line(with_lines, (y1, x1), (y2, x2), (81, 139, 255), 2)
            #------------------------#

        #------------------------#
        if (debugging):
            mostrar_img('Rectas detectadas', with_lines)
            print('Rectas detectadas ' + str(np.sum(available)))
            print('th1:' + str(third_1) + ' th2: ' + str(third_2))
        if(store):
            cv2.imwrite(path_to_imgs + file_name + '_' + 'lsl_rectas' + '.png', with_lines)
        #------------------------#

        # Selección y verificación (reglas para verificar que sean rectas válidas)
        # 1) Regularidad en el espaciado entre líneas
        # 2) No intersección dentro del ROI ni extensión por debajo de la imagen

        # -------------------------Agregar control de rectas---------------------------------------

        #-------MASCARAS-------#
        # Generación de máscaras con corrección según tipo de planta y estadío
        crop_mask = np.zeros((new_height, new_width), np.uint8)
        furrow_crop_mask = crop_mask.copy()
        x1, x2 = 0, new_height

        crop_width = int(crop_width_bottom +
                         crop_width_bottom * addition_percentage)
        crop_width_t = int(
            crop_width_top + crop_width_top * addition_percentage)
        crop_width_decay = (crop_width - crop_width_t) / x2

        # Creación de las máscaras sin superposición
        for v in range(x2 - 1, x1 - 1, -1):  # Desde la base la parte superior de la imagen
            tmp_crop_width = int(crop_width/2)
            for h in range(0, new_width):
                result = is_crop_or_furrow_lineal(
                    lines, available, (v, h), tmp_crop_width)
                if (result == 1):
                    crop_mask[v, h] = 255
                if (result == -1):
                    furrow_crop_mask[v, h] = 255
            crop_width = crop_width - crop_width_decay

        #------------------------#
        if (debugging):
            print('cw: ' + str(crop_width) + ' - ', end='')
            print('cwt: ' + str(crop_width_t))
            mostrar_imgs(['Mascara de cultivo', 'Mascara de maleza'], [
                         crop_mask, furrow_crop_mask])
        if (store):
            cv2.imwrite(path_to_imgs + file_name + '_' + 'weed_mask' + '.png', furrow_crop_mask)
        #------------------------#

        # Para la versión reducida
        furrow_crop_mask_vegetation = cv2.bitwise_and(
            furrow_crop_mask, img_veg)

        #------------------------#
        if (debugging):
            tmp = cv2.resize(original, None, fx=rs_ratio,
                             fy=rs_ratio, interpolation=cv2.INTER_AREA)
            mostrar_imgs(['Original', 'Maleza'], [
                         tmp, furrow_crop_mask_vegetation])
        if (store):
            cv2.imwrite(path_to_imgs + file_name + '_' + 'areas_para_clasificacion' + '.png', furrow_crop_mask_vegetation)
        #------------------------#

        # Disminución del contenido de vegetación en el entre surco para resaltar la maleza
#         furrow_crop_mask_vegetation = cv2.morphologyEx(furrow_crop_mask_vegetation, cv2.MORPH_OPEN, kernel_ones, iterations=1)

#         quadrants_mask = np.zeros(furrow_crop_mask.shape, np.uint8)
        # Se puede disminuir el costo verificando solo en la parte interna de la máscara de maleza, hace falta conseguir puntos que pertenecen a ella
        # Itero sobre cada franja

        total_active = np.sum(available)

        if (total_active != 0):
            for x_index in range(0, len(x_segments) - 1):  # Itera 12 veces
                # En cada franja separo en cuadrantes separados por líneas de cultivo (cantidad de líneas + 1)
                # En cada cuadrante le sumo la cantidad de vegetación y el tamaño (tamaño inicializa en 1 más para no dividir por 0)
                quadrant_total_weed = np.zeros(total_active - 1, np.uint16)
                quadrant_total_space = np.ones(total_active - 1, np.uint16)
                for x in range(x_segments[x_index + 1], x_segments[x_index]):
                    quadrant_index = 0
                    y = 0
                    # Itera máx el ancho de la imagen
                    while ((quadrant_index < len(quadrant_total_weed)) and (y < new_width)):
                        # Me saco todos los 0's del comienzo, si es que tiene
                        while ((y < new_width) and (furrow_crop_mask[x, y] == 0)):
                            y = y + 1
                        while ((y < new_width) and (furrow_crop_mask[x, y] > 0)):
                            # Procesamiento dentro de surco
                            quadrant_total_space[quadrant_index] = quadrant_total_space[quadrant_index] + 1
                            if furrow_crop_mask_vegetation[x, y] > 0:
                                quadrant_total_weed[quadrant_index] = quadrant_total_weed[quadrant_index] + 1
                            y = y + 1
                        quadrant_index = quadrant_index + 1

                # Adapto la máscara de surco acorde a si cada cuadrante pasa un porcentaje de maleza
                quadrant_total_weed = np.true_divide(
                    quadrant_total_weed, quadrant_total_space)
                for x in range(x_segments[x_index + 1], x_segments[x_index]):
                    quadrant_index = 0
                    y = 0
                    # Itera máx el ancho de la imagen
                    while ((quadrant_index < len(quadrant_total_weed)) and (y < new_width)):
                        # Me saco todos los 0's del comienzo, si es que tiene
                        while ((y < new_width) and (furrow_crop_mask[x, y] == 0)):
                            y = y + 1
                        # Mientras que esté dentro de la imagen y sea parte del entre surco
                        if (quadrant_total_weed[quadrant_index] > weed_porc):
                            # Se despinta la mascara de maleza según si el cuadrante tiene no vegetación
                            while ((y < new_width) and (furrow_crop_mask[x, y] == 255)):
                                y = y + 1
                        else:
                            while ((y < new_width) and (furrow_crop_mask[x, y] == 255)):
                                furrow_crop_mask_vegetation[x, y] = 0
                                y = y + 1
                        quadrant_index = quadrant_index + 1

        # Clasificación sectorizada
        crop_mask = cv2.resize(crop_mask, None, fx=(
            1/rs_ratio), fy=(1/rs_ratio), interpolation=cv2.INTER_LINEAR)
        if (len(lines) != 0):
            quadrants_mask = cv2.resize(furrow_crop_mask_vegetation, None, fx=(
                1/rs_ratio), fy=(1/rs_ratio), interpolation=cv2.INTER_LINEAR)
        else:
            quadrants_maskd = np.zeros(original.shape, np.int16s)
        zeros = np.zeros(crop_mask.shape, np.uint8)
        mask = cv2.merge((zeros, crop_mask, quadrants_mask))
        total = cv2.add(original, mask)

        #------------------------#
        if (debugging):
            mostrar_img('clasificada', total, reduction_to_see)
        if (store):
            cv2.imwrite(path_to_imgs + file_name + '_' + 'clasificado' + '.png', total)
        #------------------------#

